If you want to follow the following notebook, it is not enough with following the steps described at "README for fine-tuning", you need to install autogluon following the instructions on its web: https://auto.gluon.ai/stable/install.html

In [9]:
import datasets
from gluonts.dataset.arrow import ArrowWriter
from pathlib import Path
from typing import Any

In [10]:
def to_training_chronos_format(dataset: datasets.arrow_dataset.Dataset, H: int | Any = None):
    return dataset.map(
        lambda ts: 
                {'start': str(ts['timestamp'][0]), 
                 'target': ts['target'][:-H]
                 }, 
                 remove_columns=['id', 'timestamp'])

In [11]:
path = "autogluon/chronos_datasets"
datasets_with_horizons = [
                {'name': 'monash_cif_2016', 'H': 12},
                {'name': 'monash_car_parts', 'H': 12},
                {'name': 'monash_hospital', 'H': 12},
                {'name': 'monash_fred_md', 'H': 12},
                {'name': 'exchange_rate', 'H': 30},
                {'name': 'monash_covid_deaths', 'H': 30},
                {'name': 'monash_traffic', 'H': 24},
                {'name': 'monash_m3_monthly', 'H': 18},
                ]
datasets_for_finetuning = [datasets.load_dataset(path=path, name=ds['name'], split='train') for ds in datasets_with_horizons]
datasets_for_finetuning

[Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 72
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 2674
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 767
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 107
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 8
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 266
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 862
 }),
 Dataset({
     features: ['id', 'timestamp', 'target'],
     num_rows: 1428
 })]

In [12]:
for ds in datasets_for_finetuning:
    ds.set_format('numpy')

In [13]:
expected_schema = datasets.Features({
    "start": datasets.Value("string"),
    "target": datasets.Sequence(datasets.Value("float64"))
})

In [14]:
datasets_for_finetuning = list(map(
    lambda ds, ds_conf: to_training_chronos_format(ds, ds_conf['H']).cast(expected_schema), 
    datasets_for_finetuning, 
    datasets_with_horizons))
datasets_for_finetuning

[Dataset({
     features: ['start', 'target'],
     num_rows: 72
 }),
 Dataset({
     features: ['start', 'target'],
     num_rows: 2674
 }),
 Dataset({
     features: ['start', 'target'],
     num_rows: 767
 }),
 Dataset({
     features: ['start', 'target'],
     num_rows: 107
 }),
 Dataset({
     features: ['start', 'target'],
     num_rows: 8
 }),
 Dataset({
     features: ['start', 'target'],
     num_rows: 266
 }),
 Dataset({
     features: ['start', 'target'],
     num_rows: 862
 }),
 Dataset({
     features: ['start', 'target'],
     num_rows: 1428
 })]

In [15]:
datasets_for_finetuning = datasets.concatenate_datasets(datasets_for_finetuning)
datasets_for_finetuning

Dataset({
    features: ['start', 'target'],
    num_rows: 6184
})

In [16]:
ArrowWriter(compression="lz4").write_to_file(
    datasets_for_finetuning,
    path=Path.cwd()/ "training" / "data" / "training_dataset.arrow",
)

In [ ]:
!python training/train.py --config ./training/configs/chronos-t5-small-lora.yaml

In [ ]:
!python evaluation/evaluate.py evaluation/configs/zero-shot.yaml evaluation/results/chronos-t5-small-zero-shotprueba.csv --lora-model-id "output/experimental_run/checkpoint-final" --batch-size=32 --device=cuda:0 --num-samples 20